In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, QuantileTransformer, PowerTransformer, RobustScaler, MinMaxScaler
from sklearn.cluster import KMeans
import wrangle_zillow as w
import explore as e

In [2]:
df = w.get_zillow_data()
df.lotsizesquarefeet

0          4506.0
1         12647.0
2          8432.0
3         13038.0
4        278581.0
           ...   
77575     47405.0
77576     12105.0
77577      5074.0
77578      6347.0
77579      5927.0
Name: lotsizesquarefeet, Length: 77580, dtype: float64

In [3]:
df= w.single_unit_properties(df)

In [4]:
df=w.handle_missing_values(df, prop_required_column = .60, prop_required_row = .60)
df.head()

,bathroomcnt,bedroomcnt,buildingqualitytypeid,sqft,fips,fullbathcnt,latitude,longitude,lotsizesquarefeet,propertycountylandusecode,...,taxamount,logerror,transactiondate,heatingorsystemdesc,age,taxrate,acres,price_per_sqft,lotsize_per_sqft,bed_bath_ratio
0,3.5,4.0,NaN,3100.0,6059.0,3.0,33634931.0,-117869207.0,4506.0,122,...,11013.72,0.025595,2017-01-01,none,19.0,0.010763,0.103444,330.090968,227.093209,1.142857
1,1.0,2.0,NaN,1465.0,6111.0,1.0,34449266.0,-119281531.0,12647.0,1110,...,5672.48,0.055619,2017-01-01,none,50.0,0.012225,0.290335,316.723549,36.688543,2.000000
2,2.0,3.0,NaN,1243.0,6059.0,2.0,33886168.0,-117823170.0,8432.0,122,...,6488.30,0.005383,2017-01-01,none,55.0,0.011488,0.193572,454.366854,66.980313,1.500000
3,3.0,4.0,8.0,2376.0,6037.0,3.0,34245180.0,-118240722.0,13038.0,0101,...,1777.51,-0.103410,2017-01-01,Central,47.0,0.012247,0.299311,61.087121,11.132306,1.333333
6,3.0,4.0,9.0,2962.0,6037.0,3.0,34145202.0,-118179824.0,63000.0,0101,...,9516.26,-0.001011,2017-01-01,Central,67.0,0.012306,1.446281,261.074612,12.274651,1.333333


In [ ]:
train, validate, test =w.impute_missing_values(df)

In [ ]:
train, validate, test = w.impute_missing_values_1(train, validate, test)
train.head()
train.isnull().sum()

In [ ]:
train.describe().T

In [ ]:
X_train, y_train, X_validate, y_validate, X_test, y_test = w.X_train_select(train, validate, test, target_var = 'logerror')

In [ ]:
X_train.head()
X_train.describe()
X_train.shape


In [ ]:
train.shape

In [ ]:
train.info()

In [ ]:
train.isnull().sum()

In [ ]:
train.info()

In [ ]:
def add_scaled_columns(X_train, X_validate, X_test, scaler, columns_to_scale):
    """This function takes the inputs from scale_zillow and scales the data"""
    new_column_names = [c + '_scaled' for c in columns_to_scale]
    scaler.fit(X_train[columns_to_scale])

    X_train_scaled = pd.concat([
        X_train,
        pd.DataFrame(scaler.transform(X_train[columns_to_scale]), columns=new_column_names, index=X_train.index),
    ], axis=1)
    X_validate_scaled = pd.concat([
        X_validate,
        pd.DataFrame(scaler.transform(X_validate[columns_to_scale]), columns=new_column_names, index=X_validate.index),
    ], axis=1)
    X_test_scaled = pd.concat([
        X_test,
        pd.DataFrame(scaler.transform(X_test[columns_to_scale]), columns=new_column_names, index=X_test.index),
    ], axis=1)
    
    return X_train_scaled, X_validate_scaled, X_test_scaled

In [ ]:
scaler = StandardScaler()
columns_to_scale = train.drop(columns=["logerror",'propertycountylandusecode','transactiondate','heatingorsystemdesc']).columns.tolist()
X_train_scaled, X_validate_scaled, X_test_scaled = w.add_scaled_columns(X_train, X_validate, X_test, scaler, columns_to_scale)

In [ ]:
X_train_scaled

In [ ]:
e.plot_variable_pairs(train)

In [ ]:
e.plot_categorical_and_continuous_vars(cat_var, cont_var, df)